In [1]:
# Imports
import numpy as np
import picos as pc
import mosek
import numpy.linalg as la
from itertools import product
from picos import block
import qgeo as q
import math
import array
from itertools import permutations
import random
from random import randint
from picos import HermitianVariable, ComplexVariable
from scipy.stats import ortho_group
from scipy.stats import unitary_group
from matplotlib import pyplot as plt

In [3]:
# Basic functions
def permu(d,n,per):
    A = 0
    for i in product(np.arange(d),repeat=n):
        ar = list(i)
        arp = [ar[i] for i in per]
        st = ''.join(str(e) for e in ar)
        stp = ''.join(str(e) for e in arp)
        ket = q.ket(st,d)
        ketp = q.ket(stp,d)
        K = np.outer(ketp,ket)
        A += K
    return A

def Obtain_Irrep(lam,per):
    spc = s.SymmetricGroupRepresentation(lam, "specht")
    irr = spc.representation_matrix(per)
    return matrix(irr) 

def ket_from_array(arr,n):
    k=q.single_ket(int(arr[0]), d=n)
    for i in range(len(arr)):
        if i!=0:
            y=arr[i]
            k=np.kron(k, q.single_ket(int(y), d=n))
    return k

def perm_operator(arr,n,m):#matrix acting on m qu-nits which maps 012... to their permutation arr
    newshape = array.array('i',(n for i in range(2*len(arr))))
    oldaxes = array.array('i',(i for i in range(len(arr))))
    A=identity(n**m).reshape(newshape)
    B=np.moveaxis(A,arr,oldaxes).reshape(n**m,n**m)
    return(B)

def Proj_Anti(m,n):#m number of elements to permute, n dimension
    oldaxes = array.array('i',(i for i in range(0,m)))
    C=np.zeros((n**m,n**m))
    for newaxes in permutations(oldaxes):
        C+=perm_operator(newaxes,n,m)*(-1)**(is_even(newaxes)+1)
    C/=math.factorial(m)
    return(C)

def Proj_Sym(m,n):#m number of elements to permute, n dimension
    oldaxes = array.array('i',(i for i in range(0,m)))
    C=np.zeros((n**m,n**m))
    for newaxes in permutations(oldaxes):
        C+=perm_operator(newaxes,n,m)
    C/=math.factorial(m)
    return(C)

def Proj_Stan_3(n):#m3,n
    C=2*np.eye(n**3) - permu(n,3,[1,2,0]) - permu(n,3,[2,0,1])
    C=2*C#Character of identity is 2
    C/=math.factorial(3)
    return C

def PPT_entangled_edge(a,l1,l2):#A particular example of PPT ent state. Ready to use
    b=(1+a)/2
    c=np.sqrt(1-a**2)/2
    b1=a+l1*(b-a)
    b2=a+l2*(b-a)
    c1=l1*c
    c2=l2*c
    N=8*a+1+(1-a)*(l1+l2)
    a/=N
    b/=N
    c/=N
    b1/=N
    b2/=N
    c1/=N
    c2/=N
    A=np.array([[b1,c1,0,0,a,0,0,0,a],
       [c1,b1,0,0,0,0,0,0,0],
       [0,0,a,0,0,0,0,0,0],
       [0,0,0,a,0,0,0,0,0],
       [a,0,0,0,b2,c2,0,0,a],
       [0,0,0,0,c2,b2,0,0,0],
       [0,0,0,0,0,0,b,0,c],
       [0,0,0,0,0,0,0,a,0],
       [a,0,0,0,a,0,c,0,b]])
    #A/=np.trace(A)
    return A

def PPT_entangled_state_2x4(b):
    C = (1+b)/2
    D = np.sqrt(1-b**2)/2
    A=np.array([[b,0,0,0,0,b,0,0],
                [0,b,0,0,0,0,b,0],
                [0,0,b,0,0,0,0,b],
                [0,0,0,b,0,0,0,0],
                [0,0,0,0,C,0,0,D],
                [b,0,0,0,0,b,0,0],
                [0,b,0,0,0,0,b,0],
                [0,0,b,0,D,0,0,C]])/(1+7*b)
    return A

def random_mixed_state_bipartite(n):
    # generate random complex matrix A
    A = np.random.randn(n**2,n**2)#+complex(0,1)*np.random.randn(n**2,n**2)#randn(n**2,n**2)+complex(0,1)*randn(n**2,n**2)
    # compute A*A^dagger which is positive semidefinite
    A=np.matmul(A.conj().T,A)
    # normalize by trace
    A/=np.trace(A)
    return A
   
def random_mixed_state(n):
    # generate random complex matrix A
    A = np.random.randn(n,n)+complex(0,1)*np.random.randn(n,n)
    # compute A*A^dagger which is positive semidefinite
    A=np.matmul(A.conj().T,A)
    # normalize by trace
    A/=np.trace(A)
    return A

def random_mixed_state_bipartite_separable(n):
    A=random.uniform(0,1)*q.tensor_mix(random_mixed_state(n), [0], random_mixed_state(n), [1], d=n)
    r=randint(1,10000)
    for i in range(r):
        A+=random.uniform(0,1)*q.tensor_mix(random_mixed_state(n), [0], random_mixed_state(n), [1], d=n)
    A/=np.trace(A)
    return A

def GHZ_ket(n,m):
    a = np.zeros((m))
    keta=ket_from_array(a,n)
    b = np.ones((m))
    for j in range (1,n):
        keta+=ket_from_array(b,n)
        b+=np.ones((m))
    return keta

def GHZ_dens(n,m):
    return q.dm(GHZ_ket(n,m))/np.trace(q.dm(GHZ_ket(n,m)))

def identity(n):
    return np.diag(np.ones(n))

def per_12(n):
    A=identity(n**2)
    A=np.moveaxis(A.reshape(n,n,n,n),[0,1],[1,0]).reshape(n**2,n**2)
    return(A)

def is_even(p):# 0 for even permutations, 1 for odd
    if len(p) == 1:
        return True
    trans = 0
    for i in range(0,len(p)):
        j = i + 1
        for j in range(j, len(p)):
            if p[i] > p[j]:
                trans = trans + 1
    return ((trans % 2) == 0)

def singlet(sh): #Singlet with Schmidt parameter sh
    state = np.sqrt(sh)*q.ket('01')-np.sqrt(1.-sh)*q.ket('10')
    state = q.dm(state)
    return state

def Werner(v,sh):
    wer = v*singlet(sh)+(1-v)*np.eye(4)/4
    return wer

def even_numbers(numbers):
    evens=[]
    for number in numbers:
        if number %2==0:
            evens.append(number)
    return evens

def odd_numbers(numbers):
    odds=[]
    for number in numbers:
        if number %2==1:
            odds.append(number)
    return odds

def test_BInGHZ(p,par,num):
    x=np.arange(2*num)
    Rho = par*GHZ_dens(p,num)+(1-par)*np.eye(p**num)/p**4
    State = q.tensor_mix(Rho, even_numbers(x), Rho, odd_numbers(x), d=p)
    SWAP = np.eye(p**2)-2*Proj_Anti(2,p)
    WW = q.mkron_self(SWAP,num-1)
    WW = np.kron(WW,Proj_Anti(2,p))
    #WW = q.tensor_mix(SWAP, [0,1], SWAP, [2,3], SWAP, [4,5], 2*Proj_Anti(2,p), [6,7], d = [p,p,p,p,p,p,p,p])
    return np.trace(State@WW)

In [ ]:
# 4- qubit indecomposable witnesses
# For Proposition 3
def WitnessND3qubits(s,t):
    W = np.zeros((2**3,2**3))
    ar = np.array([1.,1.,-1.,1.,1.,-1.,1.,1.])
    for i in range(2**3):
        for j in range(2**3):
            if i == 2**3-j-1:
                W[i][j] = ar[i]
    W[3][3] = s
    W[4][4] = t
    return W

def PPTE3qubits(s,t):
    R = np.zeros((2**3,2**3))
    ar = np.array([-1.,-1.,1.,-1.,-1.,1.,-1.,-1.])
    for i in range(2**3):
        for j in range(2**3):
            if i == 2**3-j-1:
                R[i][j] = ar[i]
            if i == j:
                R[i][j] = 1.
    R[3][3] = s/(2.*np.sqrt(2.))
    R[4][4] = t/(2.*np.sqrt(2.))
    return R#/np.trace(R)

Wit = WitnessND3qubits(4.,2.)

Rho = PPTE3qubits(4.,2.)

print(Wit)
print(np.round(Rho,3))
print(np.trace(Wit@Rho))

X = pc.HermitianVariable("X",2**3)
P = pc.Problem(primals=True)
P.add_constraint(X >> 0)
P.add_constraint(pc.trace(X)==1.)
P.add_constraint(pc.partial_transpose(X,0,[2,2,2])>>0)
P.add_constraint(pc.partial_transpose(X,1,[2,2,2])>>0)
P.add_constraint(pc.partial_transpose(X,2,[2,2,2])>>0)
P.minimize = pc.trace(X*Wit)#X.real
# solve with mosek
P.solve(solver="mosek")
a = pc.trace(X*Wit)
Sol = X
print(a,Sol)
Evals, Evects = la.eig(Wit)
print(Evals)
Opt = np.zeros((8,8))
for i in range(8):
    if Evals[i]==-1:
        Opt += q.dm(Evects[i])
Opt /=np.trace(Opt)

from scipy.stats import ortho_group

def Setup(p,s):
    #R = PPT_entangled_edge(i/10,j/10,k/10)
    W = Wit
    #st = q.ket('100')+q.ket('010')+q.ket('001')
    #St = q.dm(st)/np.trace(q.dm(st))
    #St = GHZ_dens(2,3)*2**3
    x = ortho_group.rvs(p**3)
    St = x@GHZ_dens(2,3) # + 0.01*random_mixed_state(p**3)
    WW = q.tensor_mix(St, [0,2,4], Wit, [1,3,5], d = [p,p,p,p,p,p])#GHZ_dens(2,3)*2**3 (leads to NPT :) #np.array(Sol)#Opt#0.9*GHZ_dens(2,3)+0.1*Opt
    #x = ortho_group.rvs(p**2)
    an = q.dm(np.sqrt(s)*q.ket('00')+np.sqrt(1-s)*q.ket('11'))
    #ancilla = np.array(x@(an)@x.transpose())
    State = np.kron(identity(p**4),an*2)#Proj_Anti(2,p)#GHZ_dens(p,2)#random_mixed_state_bipartite(3)
    AAA = np.matmul(WW,State)
    aaa = np.real(q.ptrace(AAA,[4,5],d = [p,p,p,p,p,p]))
    print('hi',aaa)
    np.savetxt('witness.txt',aaa)
    return np.real(aaa)

aaa = np.real(Setup(2,0.5))
print(aaa.dtype)

print(np.any(np.iscomplex(ortho_group.rvs(8))))

def Find4partIndecWit(p,s):
    

    #X = pc.HermitianVariable("X",p**4)
    X = pc.SymmetricVariable("X",p**4)

    P = pc.Problem(primals=True)
    # add all constraints
    P.add_constraint(X >> 0)
    P.add_constraint(pc.partial_transpose(X,[0],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[1],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[2],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[3],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[0,1],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[0,2],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[0,3],[p,p,p,p])>>0)
    #P.add_constraint(pc.partial_transpose(X,[1,3],[p,p,p,p])>>0)
    #if SwapInv == 1:
    #    P.add_constraint(perm_operator([1,0],p**2,2)*X*perm_operator([1,0],p**2,2)==X)
    P.add_constraint(pc.trace(X)==1.)
    #decomp = 0.
    #count = 0
    #for i in permutations([0,1,2,3]):
    #    decomp += Z[count]*permu(p,4,i) #A += q.mkron_self(q.HW(i, j, d=p),4)*Z[i,j]
    #    count += 1
    #for i in range(p):
    #    for j in range(p):
    #        decomp += q.mkron_self(np.matmul(ket_from_array([i],p),ket_from_array([j],p).T),4)*T[i+(p*j)]
    #print(len(decomp))
    #P.add_constraint(X==decomp)         
    #add objective
    #for i in permutations([0,1,2,3]):
    #    P.add_constraint(permu(p,4,i)*X*permu(p,4,i).T==X)
    print(np.any(np.iscomplex(np.real(aaa))))
    P.minimize = pc.trace(X*np.real(aaa))#X.real
    # solve with mosek
    P.solve(solver="mosek")
    a = pc.trace(X*aaa)
    print(a)
    return X

num = Find4partIndecWit(2,0.5)
Solution = np.array(num)
print(np.round(Solution,3))
prec = 8

Sa = q.ptranspose(Solution,[0],d=[2,2,2,2])
Evs, Eves = la.eig(Sa)
print(np.round(min(Evs),prec))
Sb = q.ptranspose(Solution,[1],d=[2,2,2,2])
Evs, Eves = la.eig(Sb)
print(np.round(min(Evs),prec))
Sc = q.ptranspose(Solution,[2],d=[2,2,2,2])
Evs, Eves = la.eig(Sc)
print(np.round(min(Evs),prec))
Sd = q.ptranspose(Solution,[3],d=[2,2,2,2])
Evs, Eves = la.eig(Sd)
print(np.round(min(Evs),prec))
Sab = q.ptranspose(Solution,[0,1],d=[2,2,2,2])
Evs, Eves = la.eig(Sab)
print(np.round(min(Evs),prec))
Sac = q.ptranspose(Solution,[0,2],d=[2,2,2,2])
Evs, Eves = la.eig(Sac)
print(np.round(min(Evs),prec))
Sad = q.ptranspose(Solution,[0,3],d=[2,2,2,2])
Evs, Eves = la.eig(Sad)
print(np.round(min(Evs),prec))

# Ensure the solution is PPT(E) state
S = np.round(Solution,4)

Sa = q.ptranspose(S,[0],d=[2,2,2,2])
Evs, Eves = la.eig(Sa)
print(np.round(min(Evs),prec))
Sb = q.ptranspose(S,[1],d=[2,2,2,2])
Evs, Eves = la.eig(Sb)
print(np.round(min(Evs),prec))
Sc = q.ptranspose(S,[2],d=[2,2,2,2])
Evs, Eves = la.eig(Sc)
print(np.round(min(Evs),prec))
Sd = q.ptranspose(S,[3],d=[2,2,2,2])
Evs, Eves = la.eig(Sd)
print(np.round(min(Evs),prec))
Sab = q.ptranspose(S,[0,1],d=[2,2,2,2])
Evs, Eves = la.eig(Sab)
print(np.round(min(Evs),prec))
Sac = q.ptranspose(S,[0,2],d=[2,2,2,2])
Evs, Eves = la.eig(Sac)
print(np.round(min(Evs),prec))
Sad = q.ptranspose(S,[0,3],d=[2,2,2,2])
Evs, Eves = la.eig(Sad)
print(np.round(min(Evs),prec))


print(np.trace(S@aaa))

aaa = np.round(aaa,3)

print(aaa)

In [ ]:
# 5-qubit multipartite witnesses
# For Proposition 3

#from scipy.stats import ortho_group  
def Setup(p,s):
    #R = PPT_entangled_edge(i/10,j/10,k/10)
    W = Wit
    #st = q.ket('100')+q.ket('010')+q.ket('001')
    #St = q.dm(st)/np.trace(q.dm(st))
    #St = GHZ_dens(2,3)*2**3
    #x = ortho_group.rvs(p**3)
    St = GHZ_dens(2,4) # + 0.01*random_mixed_state(p**3)
    WW = q.tensor_mix(St, [0,2,4,6], Wit, [1,3,5], d = [p,p,p,p,p,p,p])#GHZ_dens(2,3)*2**3 (leads to NPT :) #np.array(Sol)#Opt#0.9*GHZ_dens(2,3)+0.1*Opt
    #x = ortho_group.rvs(p**2)
    an = q.dm(np.sqrt(s)*q.ket('00')+np.sqrt(1-s)*q.ket('11'))
    #ancilla = np.array(x@(an)@x.transpose())
    State = np.kron(identity(p**5),an*2)#Proj_Anti(2,p)#GHZ_dens(p,2)#random_mixed_state_bipartite(3)
    AAA = np.matmul(WW,State)
    aaa = np.real(q.ptrace(AAA,[5,6],d = [p,p,p,p,p,p,p]))
    print('hi',aaa)
    np.savetxt('witness.txt',aaa)
    return np.real(aaa)

aaa = np.real(Setup(2,0.5))


def Find5partIndecWit(p,s,l):
    

    #X = pc.HermitianVariable("X",p**4)
    X = pc.SymmetricVariable("X",p**l)

    P = pc.Problem(primals=True)
    # add all constraints
    
    ss = np.arange(l)
    ss = np.array(ss,dtype=int)
    x = len(ss)
    print(ss.dtype)
    for i in range(1 << x):
        aa = np.array([int(ss[j]) for j in range(x) if (i & (1 << j))],dtype=int)
        print(aa.dtype)
        aa = [int(el) for el in aa]
        P.add_constraint(pc.partial_transpose(X,aa,[p,p,p,p,p]) >> 0)
    #P.add_constraint(pc.partial_transpose(X,[1,3],[p,p,p,p])>>0)
    #if SwapInv == 1:
    #    P.add_constraint(perm_operator([1,0],p**2,2)*X*perm_operator([1,0],p**2,2)==X)
    P.add_constraint(pc.trace(X)==1.)
    #decomp = 0.
    #count = 0
    #for i in permutations([0,1,2,3]):
    #    decomp += Z[count]*permu(p,4,i) #A += q.mkron_self(q.HW(i, j, d=p),4)*Z[i,j]
    #    count += 1
    #for i in range(p):
    #    for j in range(p):
    #        decomp += q.mkron_self(np.matmul(ket_from_array([i],p),ket_from_array([j],p).T),4)*T[i+(p*j)]
    #print(len(decomp))
    #P.add_constraint(X==decomp)         
    #add objective
    #for i in permutations([0,1,2,3]):
    #    P.add_constraint(permu(p,4,i)*X*permu(p,4,i).T==X)
    P.minimize = pc.trace(X*np.real(aaa))#X.real
    # solve with mosek
    P.solve(solver="mosek")
    a = pc.trace(X*aaa)
    print(a)
    return X

num = Find5partIndecWit(2,0.5,5)
Solution = np.array(num)
print(np.round(Solution,3))
prec = 8

Sa = q.ptranspose(Solution,[0],d=[2,2,2,2])
Evs, Eves = la.eig(Sa)
print(np.round(min(Evs),prec))
Sb = q.ptranspose(Solution,[1],d=[2,2,2,2])
Evs, Eves = la.eig(Sb)
print(np.round(min(Evs),prec))
Sc = q.ptranspose(Solution,[2],d=[2,2,2,2])
Evs, Eves = la.eig(Sc)
print(np.round(min(Evs),prec))
Sd = q.ptranspose(Solution,[3],d=[2,2,2,2])
Evs, Eves = la.eig(Sd)
print(np.round(min(Evs),prec))
Sab = q.ptranspose(Solution,[0,1],d=[2,2,2,2])
Evs, Eves = la.eig(Sab)
print(np.round(min(Evs),prec))
Sac = q.ptranspose(Solution,[0,2],d=[2,2,2,2])
Evs, Eves = la.eig(Sac)
print(np.round(min(Evs),prec))
Sad = q.ptranspose(Solution,[0,3],d=[2,2,2,2])
Evs, Eves = la.eig(Sad)
print(np.round(min(Evs),prec))



In [4]:
# DPS to ensure the result is indeed nontrivial PPT tensor sep
# Still for Proposition 3
def DPS(dA,dB,k,R):
    R.reshape(dA*dB,dA*dB)
    d = dA
    D = dB
    X = pc.HermitianVariable("X",D*(d**(k+1)))

    P = pc.Problem(primals=True)
    # add all constraints
    P.add_constraint(X >> 0)
    for i in range(1,k+1):
        P.add_constraint(pc.partial_transpose(X,[3,3],[D*d**k,d])>>0)
        print(pc.partial_trace(X,[1],[D*d,d**k]).shape)
        R.reshape(dA*dB,dA*dB)
        print('holap')
        print(R.shape)
        #P.add_constraint(pc.partial_trace(X,[1],[D*d,d**k]) == R)
        for per in permutations(np.arange(k+1)):
            PE = permu(d,k+1,per)
            IxPE = np.kron(np.eye(D),PE)
            IxPE = np.array(IxPE, dtype=np.complex128)
            P.add_constraint(IxPE*X*IxPE.T == X)
    P.add_constraint(pc.trace(X)==1.)
    a = pc.trace(pc.partial_trace(X,[1],[D*d,d**k])*R)
    P.minimize = a
    P.solve(solver="mosek")
    print(a)
    return X

print(DPS(3,3,1,matrix))#random_mixed_state_bipartite(2)#GHZ_dens(2,2)

In [ ]:
# Detect locally bound entanglement
# For Observation 2
def Find_4partNDwit(p):
    W = Proj_Stan_3(p)/4 - Proj_Anti(3,p)
    WW = q.tensor_mix(W, [0,2,4], Proj_Anti(3,p), [1,3,5], d = [p,p,p,p,p,p])
    State = q.tensor_mix(identity(p**4), [0,1,2,3], GHZ_dens(p,2), [4,5], d = [p,p,p,p,p,p])
    AAA = np.matmul(WW,State)
    aaa = q.ptrace(AAA,[4,5],d = [p,p,p,p,p,p])
    
    X = pc.HermitianVariable("X",p**4)
    P = pc.Problem(primals=True)
    P.add_constraint(X >> 0)
    P.add_constraint(pc.trace(X) == 1.)
    P.add_constraint(pc.partial_transpose(X,0,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,1,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,2,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,3,[p,p,p,p])>>0)
    P.minimize = pc.trace(X*aaa)#X.real
    # solve with mosek
    P.solve(solver="mosek")
    a = pc.trace(X*aaa)
    print(a)
    return X

Find_4partNDwit(3)

In [5]:
# Detect not only locally PPT, but PPT across all but one bipartition 
# (not added at the article but still interesting)
def Detect_4partPPT(p,SwapInv):
    #R = PPT_entangled_edge(i/10,j/10,k/10)
    W = Proj_Stan_3(p)/4 - Proj_Anti(3,p)
    WW = q.tensor_mix(W, [0,2,4], Proj_Anti(3,p), [1,3,5], d = [p,p,p,p,p,p])
    #state = np.kron(identity(9),R)
    State = np.kron(identity(p**4),GHZ_dens(p,2))#Proj_Anti(2,p)#GHZ_dens(p,2)#random_mixed_state_bipartite(3)
    AAA = np.matmul(WW,State)
    aaa = q.ptrace(AAA,[4,5],d = [p,p,p,p,p,p])

    X = pc.SymmetricVariable("X",p**4)

    P = pc.Problem(primals=True)
    # add all constraints
    P.add_constraint(X >> 0)
    #P.add_constraint(pc.partial_transpose(X,0,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,1,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,2,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,3,[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[0,1],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[0,2],[p,p,p,p])>>0)
    P.add_constraint(pc.partial_transpose(X,[0,3],[p,p,p,p])>>0)
    if SwapInv == 1:
        P.add_constraint(perm_operator([1,0],p**2,2)*X*perm_operator([1,0],p**2,2)==X)
    P.add_constraint(pc.trace(X)==1.)
    decomp = 0.
    count = 0
    #for i in permutations([0,1,2,3]):
    #    decomp += Z[count]*permu(p,4,i) #A += q.mkron_self(q.HW(i, j, d=p),4)*Z[i,j]
    #    count += 1
    #for i in range(p):
    #    for j in range(p):
    #        decomp += q.mkron_self(np.matmul(ket_from_array([i],p),ket_from_array([j],p).T),4)*T[i+(p*j)]
    #print(len(decomp))
    #P.add_constraint(X==decomp)         
    #add objective
    #for i in permutations([0,1,2,3]):
    #    P.add_constraint(permu(p,4,i)*X*permu(p,4,i).T==X)
    P.minimize = pc.trace(X*aaa)#X.real
    # solve with mosek
    P.solve(solver="mosek")
    a = pc.trace(X*aaa)
    print(a)
    return X
A = Detect_4partPPT(3,0)
#B = Detect_4partPPT(3,1)
#Dif = A-B
#print(max(Dif))
#print(min(Dif))
#print(A)

In [ ]:
# Detect isotropic 
# For Table 1
W = Proj_Stan_3(3)/4 - Proj_Anti(3,3)
WW = q.tensor_mix(W, [0,2,4], Proj_Anti(3,3), [1,3,5], d = [3,3,3,3,3,3])

state = np.kron(identity(9),S9)
State = np.kron(state,S9)#random_mixed_state_bipartite(3) #GHZ_dens(3,2)
AAA = np.matmul(WW,State)
aaa = q.ptrace(AAA,[2,3,4,5],d = [3,3,3,3,3,3])

X = pc.SymmetricVariable("X",9,9)
P = pc.Problem(primals=True)
# add all constraints
P.add_constraint(X >> 0)
P.add_constraint(pc.partial_transpose(X,0,[3,3])>>0)
P.add_constraint(pc.partial_transpose(X,1,[3,3])>>0)
P.add_constraint(pc.trace(X)==1.)
# add objective
P.minimize = pc.trace(X*aaa)
# solve with mosek
P.solve(solver="mosek")